In [23]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.get_dataframe()

NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE = dataiku.Dataset("NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE")
NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE_df = NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE.get_dataframe()

# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

In [24]:
df = NAFCUSTOMER_CUSTOMERS_WITH_MORE_THAN_X_REVENUE_df
print(len(df))

177760


In [26]:
# find all customer_names that have other customer names that start with this customer name

import time

idx = 0
unique_customer_names = df['CUSTOMER'].unique()
full_set_n = len(unique_customer_names)

list_customers_ = []
list_potential_matches_ = []

report_every_n = 100
save_every_n = 1000

start = time.time()

for n in unique_customer_names:
    idx+=1
    df_f = df[(df['CUSTOMER'].str.startswith(n, na=False))&(df['CUSTOMER']!=n)]
    match_list = df_f['CUSTOMER'].unique()
    if len(match_list)>0:
        list_customers_.append(n)
        list_potential_matches_.append(match_list)

    if (idx % report_every_n == 0):
        print(idx, "iterations", len(list_customers_), "with potential matches", full_set_n-idx, "remaining", round((idx/full_set_n)*100,2), "% complete")

    if (idx % save_every_n == 0):
        print('SAVING DATAFRAME')
        df_candidates = pd.DataFrame(list_customers_)
        df_candidates.columns = ['CUSTOMER']
        df_candidates['POTENTIAL_MATCHES'] = list_potential_matches_

        ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df = df_candidates

        # Write recipe outputs
        ACCOUNTS_WITH_POTENTIAL_DUPLICATES = dataiku.Dataset("ACCOUNTS_WITH_POTENTIAL_DUPLICATES")
        ACCOUNTS_WITH_POTENTIAL_DUPLICATES.write_with_schema(ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df)
        print(len(ACCOUNTS_WITH_POTENTIAL_DUPLICATES_df), "written")
        
        end = time.time()
        print(round((end - start)/60,2), "minutes elapsed.")

print(len(list_customers_))

100 iterations 11 with potential matches 177660 remaining 0.06 % complete
200 iterations 26 with potential matches 177560 remaining 0.11 % complete
300 iterations 33 with potential matches 177460 remaining 0.17 % complete
400 iterations 41 with potential matches 177360 remaining 0.23 % complete
500 iterations 48 with potential matches 177260 remaining 0.28 % complete
600 iterations 60 with potential matches 177160 remaining 0.34 % complete
700 iterations 66 with potential matches 177060 remaining 0.39 % complete
800 iterations 72 with potential matches 176960 remaining 0.45 % complete
900 iterations 81 with potential matches 176860 remaining 0.51 % complete
1000 iterations 90 with potential matches 176760 remaining 0.56 % complete
SAVING DATAFRAME
90 rows successfully written (YIvUBgDGqO)
90 written
68.78549385070801 time elapsed.
1100 iterations 98 with potential matches 176660 remaining 0.62 % complete


KeyboardInterrupt: 